Install Scikit-learn into your computational methods environment with `conda install scikit-learn`

In [85]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

# Part 1: Let's get this data, fam

In [86]:
TESTING = False
data = load_breast_cancer()
n = data.data.shape[0]
valFrac = 0.2
X = data.data
Y = data.target.reshape([X.shape[0], 1])
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = valFrac, random_state = 1876)
if TESTING:
    X_train = X_train[0:10,:]
    X_val = X_train
    Y_train = Y_train[0:10]
    Y_val = Y_train
X_train = torch.from_numpy(X_train).float()
X_val = torch.from_numpy(X_val).float()
Y_train = torch.from_numpy(Y_train).float()#.long()
Y_val = torch.from_numpy(Y_val).float()#.long()
print("n Train: {}\nn Val: {}".format(X_train.shape[0], X_val.shape[0]))

n Train: 10
n Val: 10


# Part 2: Classification Functions

## Logistic Regression
Logistic regression is your bread and butter baseline classification model. Before implementing anything too fancy, you should see how far logistic regression can get you. When should you use logistic regression? Whenever you're performing supervised learning and have a binary response variable (only two classes). Luckily for us, most of the math for a logistic regression model has already been worked out in Lab 3 with linear regression. But there's just one problem, linear regression is unbounded, and if we're doing classification we want our model to predict the probability of a model belonging to one class or the other. So instead of letting our model's outputs be between $-\infty$ and $\infty$ we want to constrain it between $0$ and $1$.

To blatantly plagiarize another TAs work, in least squares linear regression we have a feature matrix $X$ and a set of corresponding outcomes $Y$, and the goal is to learn a $\beta$ such that $\hat{Y} = X^\top \beta + \epsilon$ minimizes the loss function $\sum_i (Y - \hat{Y})^2$, with $\mathbb{E}[\epsilon] = 0$.

Using the input $X$ and our model parameters $\beta$ we'll convert linear regression into logistic regression. First, why do we want to bound our model between $0$ and $1$? Because we're doing classification we need an easy way to define when our prediction is for one class or the other, and if our model can only output probabilities then we can use a cutoff (say $0.5$) and bin every observation into a class. Squashing inputs between $0$ and $1$ is done using the sigmoid function $\sigma(a) = \frac{1}{1 + exp(-a)}$. Using our inputs $X$, our learned parameters $\beta$ and $\sigma(\cdot)$ we have the makings of greatness, or at least some kind of baseline model. We write the probability of our 'positive' class (an arbiterary designation) as $$p(Y = 1|X;\beta) = \sigma(\beta^{T}X)$$ and our 'negative' class as $$p(Y = 0|X;\beta) = 1 - \sigma(\beta^{T}X)$$. For simplicity's sake let $a = beta^{T}X$ for he remainder of this cell

The loss function for logistic regression is similar to what we used in Lab 2 for MLE. For a single observation $x_i$ and its response variable $y_i$ we define our prediction's loss as $L(\beta;y_i, x_i) = \sigma(a)^{i_i}(1 - \sigma(a))^{1-y_i}$. And for an entire dataset of $n$ examples after taking the log our loss is

$$\sum_{i=1}^n y_ilog(\sigma(a)) + (1-y_i)log(1-\sigma(a))$$

Using our usual tools of gradient descent, and stochastic gradient descent we can now learn the parameters $\beta$ which minimize our loss


In [87]:
class binaryClassifier(nn.Module):
    
    # The class constructor defines the parameters (ie layers) of the neural network
    def __init__(self, nFeats, activationFunction = None):#nn.LogSoftmax(dim = 1)):
        super(binaryClassifier, self).__init__()
        # What type of parameters do we need to add?
        self.linear = nn.Linear(in_features = nFeats, out_features = 1)
        self.linear.weight= torch.nn.init.xavier_uniform_(self.linear.weight, gain = 0.001)
        self.activationFunction = activationFunction        
    # The forward method ties the layers together to build the network.
    # We take the gradient of this composite function using back propogation
    def forward(self, x):
        if self.activationFunction is None:
            out = self.linear(x)
        else:
            out = self.activationFunction(self.linear(x))
        return(out)

In [72]:
Y_hat

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]], grad_fn=<MulBackward0>)

In [107]:
normDist = torch.distributions.normal.Normal(0, 1)
probitRegressionModel = binaryClassifier(nFeats = X_train.shape[1], activationFunction = normDist.cdf)

In [74]:
probitRegressionModel.linear.weight

Parameter containing:
tensor([[ 0.0205, -0.1408,  0.0549, -0.1256,  0.1180,  0.0550, -0.0599,  0.0754,
         -0.0383, -0.0102,  0.1401, -0.1564,  0.1770,  0.0602, -0.1437, -0.1688,
          0.1500, -0.0555, -0.1377, -0.1282,  0.0701,  0.0496, -0.0673,  0.0175,
         -0.0326, -0.0566, -0.0307, -0.0974, -0.1259,  0.1155]],
       requires_grad=True)

In [87]:
torch.nn.init.xavier_uniform_(probitRegressionModel.linear.weight, gain = 0.0001)

Parameter containing:
tensor([[ 6.5663e-06, -1.9557e-05, -4.1058e-05,  2.2804e-05, -4.3453e-05,
          9.6415e-06, -1.4594e-05,  4.0654e-05,  9.1947e-06,  1.7787e-05,
          3.1158e-05, -3.9275e-06,  1.0329e-06,  3.0537e-05, -1.8159e-05,
         -2.4567e-05,  2.7381e-05,  3.4858e-06, -6.1768e-06, -6.1581e-06,
         -9.5324e-06,  4.1520e-05, -2.1943e-05,  4.1130e-05,  1.3174e-05,
          3.6454e-05,  8.1693e-06,  1.2613e-05, -1.6743e-05,  3.1575e-05]],
       requires_grad=True)

In [109]:
m = nn.Sigmoid()
logisticRegressionModel = binaryClassifier(nFeats = X_train.shape[1], activationFunction = None)
lossFunction = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(logisticRegressionModel.parameters(), lr = .0001, momentum = 0.9)

In [110]:
epochs = 1000

for epoch in range(epochs):
        
    # Estimate Y_hat with the current model
    
    Y_hat = logisticRegressionModel(X_train)
    
    # Compute the loss
    loss = lossFunction(Y_hat, Y_train)
        
    # Compute the gradient of the loss
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print("loss {}".format(loss))

loss 6.628029823303223
loss 0.00011123196600237861
loss 5.960463411724959e-08
loss 5.960463411724959e-08
loss 5.960463411724959e-08
loss 5.960463411724959e-08
loss 5.960463411724959e-08
loss 5.960463411724959e-08
loss 5.960463411724959e-08
loss 5.960463411724959e-08


## Probit Model

We defined $p(Y = 1|a) = \sigma(a)$ in logistic regression, but the more general form of a linear model would be $p(Y = 1|X;\beta) = f(\beta^{T}X)$ where $f(\cdot)$ is known as an activation function. Another activation function we could have used is known as the inverse probit function which is the cumulative distribution function of a standard normal defined as $\Phi(a) = \frac{1}{2}(1 + erf(\frac{1}{\sqrt{2}}))$ (Did you hear that? That was the sound of an absolute ton of details being skipped over. For more information about probit regression on page 210 [here](http://users.isr.ist.utl.pt/~wurmd/Livros/school/Bishop%20-%20Pattern%20Recognition%20And%20Machine%20Learning%20-%20Springer%20%202006.pdf)). Here $erf(\cdot)$ is known as the error function. All the same steps apply for the logistic regression example, except instead of $\sigma(\cdot)$ we use $\Phi(\cdot)$

$$\sum_{i=1}^n y_ilog(\Phi(a)) + (1-y_i)log(1-\Phi(a))$$



Tips:
1. For the love of your weekend don't try to implement the CDF of a normal distribution. Search around and find out how you can get the cdf of different distributions in using Pytorch.

In [73]:
normDist = torch.distributions.normal.Normal(0, 1)
probitRegressionModel = binaryClassifier(nFeats = X_train.shape[1], activationFunction = normDist.cdf)
lossFunction = nn.BCELoss()
optimizer = torch.optim.SGD(probitRegressionModel.parameters(), lr = .000001, momentum = 0.9)

In [74]:
epochs = 1000

for epoch in range(epochs):
        
    # Estimate Y_hat with the current model
    
    Y_hat = probitRegressionModel(X_train)
    
    # Compute the loss
    loss = lossFunction(Y_hat, Y_train)
        
    # Compute the gradient of the loss
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print("loss {}".format(loss))

loss 0.7132978439331055
loss 0.32142892479896545
loss 0.21147653460502625
loss 0.15404944121837616
loss 0.11979521811008453
loss 0.09743179380893707
loss 0.08183403313159943
loss 0.07039885222911835
loss 0.061685435473918915
loss 0.05484015494585037


## Hinge Loss
We can also change the loss function we use in logistic regression to the hinge loss by reconfiguring how we view the data. To do this we formulate our response variable as either $-1$ or $1$. Now, $p(Y = 1|X;\beta) = \sigma(a)$ remains unchanged, but $p(Y = -1|X;\beta) = 1 - \sigma(a) = \sigma(-a) = \sigma(ya)$. In the last step recall that $Y \in \{-1, 1\}$ so $p(Y|X;\beta) = \sigma(ya)$

Using the log likelihood and our updated probability functions our loss now becomes:

$$\sum_{i=1}^{n}\sigma(ya)$$

In [118]:
def myHingeLoss(a, y):
    m = nn.LogSigmoid()
    out = -m(Y_hat*Y_train_hinge)
    out = torch.mean(out)
    return(out)

In [119]:
logisticRegressionHingeModel = binaryClassifier(nFeats = X_train.shape[1], activationFunction = None)
lossFunction = myHingeLoss
optimizer = torch.optim.SGD(logisticRegressionHingeModel.parameters(), lr = .0001, momentum = 0.9)

In [120]:
Y_train_hinge = Y_train.clone().detach()
Y_train_hinge[Y_train_hinge == 0] = -1

In [121]:
epochs = 1000
for epoch in range(epochs):
        
    # Estimate Y_hat with the current model
    
    Y_hat = logisticRegressionHingeModel(X_train)
    
    # Compute the loss
    loss = lossFunction(Y_hat,Y_train_hinge)
        
    # Compute the gradient of the loss
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print("loss {}".format(loss))

loss 0.7578908205032349
loss 1.4084984064102173
loss 7.104622000042582e-06
loss 3.7550223623838974e-06
loss 2.6464110760571202e-06
loss 2.0503787254710915e-06
loss 1.6689160702298977e-06
loss 1.4185804957378423e-06
loss 1.227848088092287e-06
loss 1.084798668671283e-06


In [136]:
# Y_hat
m = nn.Sigmoid()
m(Y_hat)

tensor([[1.0000e+00],
        [1.0000e+00],
        [1.0000e+00],
        [1.1202e-24],
        [0.0000e+00],
        [1.4829e-11],
        [2.9223e-10],
        [1.0000e+00],
        [1.0000e+00],
        [9.9999e-01]], grad_fn=<SigmoidBackward>)

In [135]:
Y_train

tensor([[1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.]])

In [129]:
Y_hat

tensor([[  17.4905],
        [  26.0904],
        [  30.7312],
        [ -55.1486],
        [-119.8049],
        [ -24.9344],
        [ -21.9535],
        [  30.1257],
        [  32.1253],
        [  11.5451]], grad_fn=<AddmmBackward>)

In [96]:
Y_train_hinge

tensor([[ 1.],
        [ 1.],
        [ 1.],
        [-1.],
        [-1.],
        [-1.],
        [-1.],
        [ 1.],
        [ 1.],
        [ 1.]])

# Part 3 Outliers